In [3]:
import numpy as np
from numba import njit
from functools import partial
import plotly.graph_objects as go
import talib 
import pandas as pd
import vectorbt as vbt
import vectorbt as vbt
from vectorbt.utils.colors import adjust_opacity
from vectorbt.utils.enum_ import map_enum_fields
from vectorbt.base.reshape_fns import broadcast, flex_select_auto_nb, to_2d_array
from vectorbt.portfolio.enums import SizeType, Direction, NoOrder, OrderStatus, OrderSide
from vectorbt.portfolio import nb
import sys
import os
function_essential_path = r"C:\Users\Jordi\Desktop\Environement de developement\Trading_Dev_Stratégie_Environement\FunctionEssential"
function_s_score_path = r'C:\Users\Jordi\Desktop\Environement de developement\Trading_Dev_Stratégie_Environement\Indicateur'
sys.path.append(function_essential_path)
sys.path.append(function_s_score_path)
import utils as us
import s_score as score

In [12]:
##FONCTION LOGIQUE SIGNALS##

##FONCTION LOGIQUE SIGNALS##

def entries_trades(close,sscore,slope,bb_up,bb_low,seuil_score,seuil_slope):
    long = np.full_like(close, False, dtype=bool)
    short = np.full_like(close, False, dtype=bool)
    trade_out = True

    for col in range(sscore.shape[1]):
        for idx in range(len(close)):

            #if (slope[idx, col] <= seuil_slope) & (slope[idx, col] >= -seuil_slope):
            if trade_out:
                if sscore[idx, col] <= -seuil_score:
                    #if bb_low[idx, col] >= close[idx, col]:
                        long[idx, col] = True
                        trade_out = False
                if sscore[idx, col] >= seuil_score:
                    #if bb_up[idx, col] <= close[idx, col]:
                        short[idx, col] = True
                        trade_out = False
            else:
                if (sscore[idx, col] <= 0.5) & (sscore[idx, col] >= -0.5):
                    trade_out = True

    return long, short




def exits_trades(long,short,close,sscore,seuil_sscore_out,seuil_sl):
    long_exit = np.zeros_like(long, dtype=np.float64)
    short_exit = np.zeros_like(short, dtype=np.float64)
    for col in range(long.shape[1]):
        for idx in range(len(close)):
            if long[idx, col]:
                SL = close[idx, col] * (1-seuil_sl)
                for j in range(idx+1,len(close)):
                    if close[j, col] <= SL:
                        long_exit[j, col] = SL
                        break
                    if sscore[j, col] >= -seuil_sscore_out:
                        long_exit[j, col] = close[j, col]
                        break

            if short[idx, col]:

                SL = close[idx, col] * (1+seuil_sl)
                for j in range(idx+1,len(close)):

                    if close[j, col] >= SL:
                        short_exit[j, col] = SL
                        break

                    if sscore[j, col] <= seuil_sscore_out:
                        short_exit[j, col] = close[j, col]
                        break

    return long_exit,short_exit

# def exits_trades(long,short,close,sscore,seuil_sscore_out,seuil_sl):
#     long_exit = np.zeros_like(long, dtype=np.float64)
#     short_exit = np.zeros_like(short, dtype=np.float64)
#     for idx in range(len(close)):
#         if long[idx]:
#             SL = close[idx] * (1-seuil_sl)
#             for j in range(idx+1,len(close)):
#                 if close[j] <= SL:
#                     long_exit[j] = SL
#                     break
#                 if sscore[j] >= -seuil_sscore_out:
#                     long_exit[j] = close[j]
#                     break

#         if short[idx]:

#             SL = close[idx] * (1+seuil_sl)
#             for j in range(idx+1,len(close)):

#                 if close[j] >= SL:
#                     short_exit[j] = SL
#                     break

#                 if sscore[j] <= seuil_sscore_out:
#                     short_exit[j] = close[j]
#                     break
                    
    # return long_exit,short_exit


In [13]:
##INDICATEUR FACTORY##

Entry = vbt.IndicatorFactory(
    class_name="entries_trades",
    input_names=["close","sscore","slope","bb_up","bb_low"],
    param_names=["seuil_score","seuil_slope"],
    output_names=["long","short"],
).from_apply_func(entries_trades)

Exit = vbt.IndicatorFactory(
    class_name="exits_trades",
    input_names=["long","short","close","sscore"],
    param_names=["seuil_sscore_out","seuil_sl"],
    output_names=["long","short"],
).from_apply_func(exits_trades)

In [14]:
@njit
def pre_sim_func_nb(c):
    entry_price = np.full(c.target_shape[1], np.nan, dtype=np.float64)  
    return (entry_price,)

@njit
def order_func_nb(c, entry_price, long, short, long_exit,short_exit, size):
    price_now = nb.get_elem_nb(c, c.close)
    entry_long_now = nb.get_elem_nb(c, long)
    entry_short_now = nb.get_elem_nb(c, short)


    if entry_long_now:
        if c.position_now == 0:
            return nb.order_nb(
                size[c.col], 
                price=price_now,
                direction=Direction.LongOnly,
                fees=0.001
            )
        elif c.position_now < 0:
            return nb.order_nb(
                -size[c.col],  
                price=price_now,
                direction=Direction.ShortOnly,
                fees=0.0005
            )  

    if entry_short_now:
        if c.position_now == 0:
            return nb.order_nb(
                size[c.col], 
                price=price_now,
                direction=Direction.ShortOnly,
                fees=0.001
            )
        elif c.position_now > 0:
            return nb.order_nb(
                -size[c.col],  
                price=price_now,
                direction=Direction.LongOnly,
                fees=0.0005
            )

    #Si la valeur de l'idx de la sortie est différente de 0 ca veut dire que on doit out et on récupere la valeur de notre np.array long_exit 
    if c.position_now > 0:
        long_exit_now = nb.get_elem_nb(c, long_exit)
        if long_exit_now != 0:
            return nb.order_nb(
                -size[c.col],
                price=long_exit_now,
                direction=Direction.LongOnly,
                fees=0.001
            )

    if c.position_now < 0:
        short_exit_now = nb.get_elem_nb(c, short_exit)
        if short_exit_now != 0:
            return nb.order_nb(
                -size[c.col],
                price=short_exit_now,
                direction=Direction.ShortOnly,
                fees=0.001
            )
       

    return NoOrder

@njit
def post_order_func_nb(c, entry_price):
    if c.order_result.status == OrderStatus.Filled:
        entry_price[c.col] = c.order_result.price

In [ ]:
##CLASS STRATEGIE##

class Strategie():

    def __init__(self,data,tickers,frequence):
        self.data = data
        self.tickers = tickers
        self.frequences = frequence


    def set_params_strategie(self,sl,seuil_score,out,slope_range,sscore,period_ema,period_bb,mf):
        self.stop_loss = sl
        self.seuil_score = seuil_score
        self.ssscore_out = out
        self.slope_range = slope_range
        self.sscore = sscore
        self.period_ema = period_ema
        self.period_bb = period_bb
        self.main_frequences = mf


    def backtest(self):

            entries_long_df = pd.DataFrame()   
            entries_short_df = pd.DataFrame()
            exits_long_df = pd.DataFrame()
            exits_short_df = pd.DataFrame()
            close_df = pd.DataFrame()
            datetime_df = pd.DataFrame()

            for ticker in self.tickers:

                data = self.data[self.main_frequences][ticker]
                datetime = data.reset_index()['Datetime']

                ema = talib.EMA(data['close'], timeperiod=self.period_ema)
                slope = np.gradient(ema)  
                s_score_ajuster= self.sscore[ticker]['s_score_value']

                bb = vbt.BBANDS.run(data['close'], window=self.period_bb, short_name='BB')

                # Extraire les bandes supérieure, moyenne et inférieure
                upper_band = bb.upper
                lower_band = bb.lower

                trades = Entry.run(
                    data['close'],
                    s_score_ajuster,
                    slope,
                    upper_band,
                    lower_band,
                    self.seuil_score,
                    self.slope_range
                )

                exit = Exit.run(
                    trades.long,
                    trades.short,
                    data['close'],
                    s_score_ajuster,
                    self.ssscore_out,
                    self.stop_loss
                )

                us.print_trades(trades.long,trades.short,None,None,data)

                entries_long_df[ticker] = trades.long.reset_index(drop=True)
                entries_short_df[ticker] = trades.short.reset_index(drop=True)
                exits_long_df[ticker] = exit.long.reset_index(drop=True)
                exits_short_df[ticker] = exit.short.reset_index(drop=True)
                close_df[ticker] = data['close'].reset_index(drop=True)
                datetime_df[ticker] = datetime.reset_index(drop=True)

                

            size = np.array([600000]*len(self.tickers))
            exits_long_df = exits_long_df.astype(np.float64).fillna(0)
            exits_short_df = exits_short_df.astype(np.float64).fillna(0)
            s_score_ajuster = np.asarray(s_score_ajuster, dtype=np.float64).reshape(-1,1)
            
            portfolio = vbt.Portfolio.from_order_func(
                close_df,  
                order_func_nb,  
                entries_long_df.values,  
                entries_short_df.values,  
                exits_long_df.values,
                exits_short_df.values,
                size,  
                freq=self.frequences,  
                init_cash=100000,  # Ajout du capital initial
                pre_sim_func_nb=pre_sim_func_nb,  
                post_order_func_nb=post_order_func_nb,  
                broadcast_named_args=dict(  
                    long=entries_long_df.values,  
                    short=entries_short_df.values,  
                    exits_long_df=exits_long_df.values,  
                    exits_short_df=exits_short_df.values,  
                    size=size  
                )
            )
            us.rapport_backtest(portfolio,close_df,datetime_df,self.tickers)
            return portfolio

    def optimize(self,fitness_func,close,s_score_ajuster,choice,params1,params2):
         
        if choice == 1:
            
            combinaison=[]
            for i in params1:
                for j in params2:
                    combinaison.append((i,j))

            ema = vbt.MA.run(close, window=params1, short_name='EMA').ma

            slope_dict = {}  # Dictionnaire pour stocker les résultats

            for col in ema.columns:
                slope_dict[col] = np.gradient(ema[col])  # Calcul de la pente

            # Conversion en DataFrame
            slope = pd.DataFrame(slope_dict)

            bb = vbt.BBANDS.run(close, window=self.period_bb, short_name='BB')

            # Extraire les bandes supérieure, moyenne et inférieure
            upper_band = bb.upper
            lower_band = bb.lower


            #us.get_details_variable([close,slope,s_score_ajuster,self.seuil_score,params2])
            trades = Entry.run(
                close,
                s_score_ajuster,
                slope,
                upper_band,
                lower_band,
                self.seuil_score,
                params2
            )

            exit = Exit.run(
                trades.long,
                trades.short,
                close,
                s_score_ajuster,
                self.ssscore_out,
                self.stop_loss
            )

            long = trades.long
            short = trades.short
            long_exit = exit.long
            short_exit = exit.short


            long = long.reset_index(drop=True)
            short = short.reset_index(drop=True)
            long_exit = long_exit.reset_index(drop=True)
            short_exit = short_exit.reset_index(drop=True)
            
            close = close.reset_index(drop=True)
            close = pd.DataFrame(close)  

            capital = 100000  
            size = np.array([(0.05 * capital) / (0.02 * close.iloc[0, i]) for i in range(close.shape[1])])
            size = np.array([size[0]] * long.shape[1])

            
            us.get_details_variable([long,short,long_exit,short_exit,size])
            portfolio = vbt.Portfolio.from_order_func(
                close,  
                order_func_nb,  
                long.values,  
                short.values,  
                long_exit.values,
                short_exit.values,
                size,  
                freq=self.frequences,  
                init_cash=100000,  
                pre_sim_func_nb=pre_sim_func_nb,  
                post_order_func_nb=post_order_func_nb,  
                broadcast_named_args=dict(  
                    long=long.values,  
                    short=short.values,  
                    exits_long_df=long_exit.values,  
                    exits_short_df=short_exit.values,  
                    size=size  
                )
            )
            

            perf = us.get_heatmap(portfolio,combinaison,fitness_func)
            us.get_3d_surface_metrics(2,portfolio,combinaison,fitness_func,"seuil_out","sl %",30,90)
            param1,param2,metrics = us.get_best_param(perf)
            return param1,param2
        
        if choice == 2:
            


            ema = vbt.MA.run(close, window=self.period_ema, short_name='EMA').ma
            slope_dict = np.gradient(ema)  


            # Conversion en DataFrame
            slope = pd.DataFrame(slope_dict)
            bb = vbt.BBANDS.run(close, window=self.period_bb, short_name='BB')

            # Extraire les bandes supérieure, moyenne et inférieure
            upper_band = bb.upper
            lower_band = bb.lower


            #us.get_details_variable([close,slope,s_score_ajuster,self.seuil_score,params2])
            trades = Entry.run(
                close,
                s_score_ajuster,
                slope,
                upper_band,
                lower_band,
                params1,
                self.slope_range
            )

            exit = Exit.run(
                trades.long,
                trades.short,
                close,
                s_score_ajuster,
                self.ssscore_out,
                self.stop_loss
            )

            long = trades.long
            short = trades.short
            long_exit = exit.long
            short_exit = exit.short


            long = long.reset_index(drop=True)
            short = short.reset_index(drop=True)
            long_exit = long_exit.reset_index(drop=True)
            short_exit = short_exit.reset_index(drop=True)
            
            close = close.reset_index(drop=True)
            close = pd.DataFrame(close)  

            capital = 100000  
            size = np.array([(0.05 * capital) / (0.02 * close.iloc[0, i]) for i in range(close.shape[1])])
            size = np.array([size[0]] * long.shape[1])

            
            us.get_details_variable([long,short,long_exit,short_exit,size])
            portfolio = vbt.Portfolio.from_order_func(
                close,  
                order_func_nb,  
                long.values,  
                short.values,  
                long_exit.values,
                short_exit.values,
                size,  
                freq=self.frequences,  
                init_cash=100000,  
                pre_sim_func_nb=pre_sim_func_nb,  
                post_order_func_nb=post_order_func_nb,  
                broadcast_named_args=dict(  
                    long=long.values,  
                    short=short.values,  
                    exits_long_df=long_exit.values,  
                    exits_short_df=short_exit.values,  
                    size=size  
                )
            )
            

            perf = us.get_barplot(portfolio,params1,fitness_func)
            # us.get_3d_surface_metrics(2,portfolio,combinaison,fitness_func,"seuil_out","sl %",30,90)
            param1,param2,metrics = us.get_best_param(perf)
            return param1,param2
        if choice == 3:
            
            combinaison=[]
            for i in params1:
                for j in params2:
                    combinaison.append((i,j))

            ema = vbt.MA.run(close, window=self.period_ema, short_name='EMA').ma
            slope = np.gradient(ema)  
            slope = pd.DataFrame(slope)
            bb = vbt.BBANDS.run(close, window=self.period_bb, short_name='BB')

            # Extraire les bandes supérieure, moyenne et inférieure
            upper_band = bb.upper
            lower_band = bb.lower



            trades = Entry.run(
                close,
                s_score_ajuster,
                slope,
                upper_band,
                lower_band,
                self.seuil_score,
                self.slope_range
            )

            exit = Exit.run(
                trades.long,
                trades.short,
                close,
                s_score_ajuster,
                params2,
                params1,
                param_product=True
            )

            long = trades.long
            short = trades.short
            long_exit = exit.long
            short_exit = exit.short


            long = long.reset_index(drop=True)
            short = short.reset_index(drop=True)
            long_exit = long_exit.reset_index(drop=True)
            short_exit = short_exit.reset_index(drop=True)
            
            close = close.reset_index(drop=True)
            close = pd.DataFrame(close)  

            capital = 100000  
            size = np.array([(0.05 * capital) / (0.02 * close.iloc[0, i]) for i in range(close.shape[1])])
            size = np.array([size[0]] * long_exit.shape[1])

            
            us.get_details_variable([long,short,long_exit,short_exit,size])
            portfolio = vbt.Portfolio.from_order_func(
                close,  
                order_func_nb,  
                long.values,  
                short.values,  
                long_exit.values,
                short_exit.values,
                size,  
                freq=self.frequences,  
                init_cash=100000,  
                pre_sim_func_nb=pre_sim_func_nb,  
                post_order_func_nb=post_order_func_nb,  
                broadcast_named_args=dict(  
                    long=long.values,  
                    short=short.values,  
                    exits_long_df=long_exit.values,  
                    exits_short_df=short_exit.values,  
                    size=size  
                )
            )
            

            perf = us.get_heatmap(portfolio,combinaison,fitness_func)
            us.get_3d_surface_metrics(2,portfolio,combinaison,fitness_func,"seuil_out","sl %",30,90)
            param1,param2,metrics = us.get_best_param(perf)
            return param1,param2
        
        if choice == 4:
            

            combinaison=[]
            for i in range(0,23):
                for j in params2:
                    name_column = params1.columns[i]
                    combinaison.append((name_column,j))





            ema = vbt.MA.run(close, window=self.period_ema, short_name='EMA').ma
            slope_dict = np.gradient(ema)  


            # Conversion en DataFrame
            slope = pd.DataFrame(slope_dict)
            bb = vbt.BBANDS.run(close, window=self.period_bb, short_name='BB')

            # Extraire les bandes supérieure, moyenne et inférieure
            upper_band = bb.upper
            lower_band = bb.lower


            #us.get_details_variable([close,slope,s_score_ajuster,self.seuil_score,params2])
            trades = Entry.run(
                close,
                params1,
                slope,
                upper_band,
                lower_band,
                params2,
                self.slope_range
            )

            exit = Exit.run(
                trades.long,
                trades.short,
                close,
                params1,
                self.ssscore_out,
                self.stop_loss
            )

            long = trades.long
            short = trades.short
            long_exit = exit.long
            short_exit = exit.short


            long = long.reset_index(drop=True)
            short = short.reset_index(drop=True)
            long_exit = long_exit.reset_index(drop=True)
            short_exit = short_exit.reset_index(drop=True)
            
            close = close.reset_index(drop=True)
            close = pd.DataFrame(close)  

            capital = 100000  
            size = np.array([(0.05 * capital) / (0.02 * close.iloc[0, i]) for i in range(close.shape[1])])
            size = np.array([size[0]] * long.shape[1])

            
            us.get_details_variable([long,short,long_exit,short_exit,size])
            portfolio = vbt.Portfolio.from_order_func(
                close,  
                order_func_nb,  
                long.values,  
                short.values,  
                long_exit.values,
                short_exit.values,
                size,  
                freq=self.frequences,  
                init_cash=100000,  
                pre_sim_func_nb=pre_sim_func_nb,  
                post_order_func_nb=post_order_func_nb,  
                broadcast_named_args=dict(  
                    long=long.values,  
                    short=short.values,  
                    exits_long_df=long_exit.values,  
                    exits_short_df=short_exit.values,  
                    size=size  
                )
            )
            

            perf = us.get_heatmap(portfolio,combinaison,fitness_func)
            us.get_3d_surface_metrics(2,portfolio,combinaison,fitness_func,"Period S-score","s-score seuil",30,90)
            param1,param2,metrics = us.get_best_param(perf)
            return param1,param2


    def wfa(self,period_OOS, period_IOS):
 
        close_df = pd.DataFrame()
        for ticker in self.tickers:

            data = self.data[ticker]
            cycle = us.generate_wf_cycles(data, period_IOS, period_OOS)
            close = data['close']
            close_df[ticker] = close

            for i in range(len(cycle)):

                close_IOS = data["close"].iloc[cycle[i][0]:cycle[i][1]]
                close_OOS = data["close"].iloc[cycle[i][2]:cycle[i][3]]

                

                params1 = np.arange(10, 16, 1)
                params2 = np.arange(15, 26, 1)
                params1_opti, params2_opti = self.optimize()
                
                params3 = np.round(np.arange(0.5, -0.6, -0.1), 1)
                params4 = np.arange(0.009, 0.016, 0.001)
                params3_opti, params4_opti = self.optimize()

                #premiere itération on save la période IOS pour afficher l'entierté de mon jeu de donnée 
                if i == 0:

                    trades = Entry.run(
                        )
                    
                    exit = Exit.run(
                    )

                    entries_long = trades.long.to_frame(name=ticker)
                    entries_short = trades.short.to_frame(name=ticker)
                    exits_long = exit.long.to_frame(name=ticker)
                    exits_short = exit.short.to_frame(name=ticker)


                trades = Entry.run(

                )
                
                exit = Exit.run(
                    )

                entries_long = pd.concat([entries_long, trades.long.to_frame(name=ticker)], ignore_index=True)
                entries_short = pd.concat([entries_short, trades.short.to_frame(name=ticker)], ignore_index=True)
                exits_long = pd.concat([exits_long, exit.long.to_frame(name=ticker)], ignore_index=True)
                exits_short = pd.concat([exits_short, exit.short.to_frame(name=ticker)], ignore_index=True)




            capital = 100000   
            size = np.array([(0.05 * capital) / (0.02 * close_df.iloc[0, i]) for i in range(len(self.tickers))])
            stop_loss = np.array([0.01] * len(self.tickers))  
            take_profit = np.array([0.02] * len(self.tickers))

            portfolio = vbt.Portfolio.from_order_func(
                close,  
                order_func_nb,  
                entries_long.values,  
                entries_short.values,  
                exits_long.values,
                exits_short.values,
                size,  
                freq=self.frequences,  
                init_cash=100000,  # Ajout du capital initial
                pre_sim_func_nb=pre_sim_func_nb,  
                post_order_func_nb=post_order_func_nb,  
                broadcast_named_args=dict(  
                    long=entries_long.values,  
                    short=entries_short.values,  
                    exits_long=exits_long.values,  
                    exits_short=exits_short.values,  
                    size=size  
                )
            )
            us.get_pnl(portfolio)
            return portfolio



In [58]:
# tickers = ['GPDUSD', 'EURUSD']
# DATA = us.get_data_forex(tickers, ['H4'], "2003-01-01", "2017-01-01")

# adjusted_s_scores = {ticker: {} for ticker in tickers}  # Stocke les valeurs ajustées
# s_scores_no_adjusted = {ticker: {} for ticker in tickers}  # Stocke les scores bruts

# period_range = np.arange(60, 105, 2)

# for ticker in tickers:
#     data_traitement = DATA['H4'][ticker]
#     nb_nan = data_traitement['close'].isna().sum()
#     close = data_traitement['close'].dropna()
#     close_without_index = close.reset_index()

#     for period in period_range:
#         result = score.mean_reversion(close, close_without_index['Datetime'], period, "VOL", 0.5, "EGARCH")
        
#         adjusted_s_scores_vol = result[1]  # adjusted_s_scores_vol
#         s_scores_raw = result[0]  # s_score_no_adjusted
        
#         # Ajouter des NaN pour compenser les données manquantes
#         adjusted_s_scores_vol = np.concatenate([adjusted_s_scores_vol, [np.nan] * nb_nan])
#         s_scores_raw = np.concatenate([s_scores_raw, [np.nan] * nb_nan])

#         # Stocker correctement dans les dictionnaires
#         adjusted_s_scores[ticker][period] = adjusted_s_scores_vol  
#         s_scores_no_adjusted[ticker][period] = s_scores_raw  

# # Transformer en DataFrame pour obtenir la bonne shape (X, Y)
# df_adjusted_s_scores_EURUSD = pd.DataFrame(adjusted_s_scores['EURUSD'])
# df_adjusted_s_scores_GPDUSD = pd.DataFrame(adjusted_s_scores['GPDUSD'])

# df_s_scores_EURUSD = pd.DataFrame(s_scores_no_adjusted['EURUSD'])
# df_s_scores_GPDUSD = pd.DataFrame(s_scores_no_adjusted['GPDUSD'])


tickers = ['EURUSD','GPDUSD']
DATA = us.get_data_forex(tickers,['M15'],"2003-01-01","2020-01-01")
s_score_value = {}
for ticker in tickers:
    data_traitement = DATA['M15'][ticker]
    nb_nan = data_traitement['close'].isna().sum()
    close = data_traitement['close'].dropna()
    close_without_index = close.reset_index()
    result = score.mean_reversion(close, close_without_index['Datetime'], 85, "VOL", 0.5, "EGARCH")
    result_df = pd.DataFrame({
        's_score_value': result[0],
        'adjusted_s_scores_vol': result[1],
        'theta_vals': result[2],
        'mu_vals': result[3],
        'sigma_vals': result[4]
    })
    nan_rows = pd.DataFrame([[float('nan')] * result_df.shape[1]] * nb_nan, columns=result_df.columns)
    result_df = pd.concat([result_df, nan_rows], ignore_index=True)

    s_score_value[ticker] = result_df


C:\Users\Jordi\Desktop\Environement de developement\Trading_Dev_Stratégie_Environement\FunctionEssential\utils.py:244: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.

C:\Users\Jordi\Desktop\Environement de developement\Trading_Dev_Stratégie_Environement\FunctionEssential\utils.py:244: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.

Estimation des paramètres OU:   0%|          | 0/431981 [00:00<?, ?it/s]

  0%|          | 0/431981 [00:00<?, ?it/s]

Estimation des paramètres OU:   0%|          | 0/108351 [04:38<?, ?it/s]
C:\Users\Jordi\Desktop\Environement de developement\Trading_Dev_Stratégie_Environement\Indicateur\s_score.py:31: RuntimeWarning:

invalid value encountered in sqrt

c:\Users\Jordi\AppData\Local\Programs\Python\Python312\Lib\site-packages\arch\univariate\base.py:768: ConvergenceWarning:

The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.


Calcul de la volatilité conditionnelle: 100%|██████████| 432066/432066 [00:01<00:00, 261143.22it/s]


432065


C:\Users\Jordi\AppData\Local\Temp\ipykernel_14988\2259273512.py:54: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

Estimation des paramètres OU:   0%|          | 0/431722 [00:00<?, ?it/s]

  0%|          | 0/431722 [00:00<?, ?it/s]

C:\Users\Jordi\Desktop\Environement de developement\Trading_Dev_Stratégie_Environement\Indicateur\s_score.py:31: RuntimeWarning:

invalid value encountered in sqrt

c:\Users\Jordi\AppData\Local\Programs\Python\Python312\Lib\site-packages\arch\univariate\base.py:768: ConvergenceWarning:

The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.


Calcul de la volatilité conditionnelle: 100%|██████████| 431807/431807 [00:01<00:00, 223991.41it/s]


431806


In [1]:
strat = Strategie(DATA,tickers,"15m")

strat.set_params_strategie(0.005,3,0,0.005,s_score_value,5,20,'M15')
portfolio = strat.backtest()


s_score = np.round(np.arange(2,4,0.1),1)
strat.optimize(2,DATA['M15']['EURUSD']['close'],s_score_value['EURUSD']['s_score_value'],2,s_score,None)
strat.optimize(2,DATA['M15']['GPDUSD']['close'],s_score_value['GPDUSD']['s_score_value'],2,s_score,None)

# strat.optimize(2,DATA['H4']['EURUSD']['close'],s_score_value['EURUSD']['adjusted_s_scores_vol'],2,s_score,None)
# strat.optimize(2,DATA['H4']['GPDUSD']['close'],s_score_value['GPDUSD']['adjusted_s_scores_vol'],2,s_score,None)
# strat.optimize(2,DATA['H4']['GPDUSD']['close'],None,4,df_adjusted_s_scores_GPDUSD,s_score)
# strat.optimize(2,DATA['H4']['EURUSD']['close'],None,4,df_adjusted_s_scores_EURUSD,s_score)
# ema_period = np.arange(2,33,1)
# slope_range = np.arange(0.0001,0.001,0.0001)
# s_score = np.round(np.arange(2,4,0.1),1)
# stop_loss = np.round(np.arange(0.005,0.02,0.001),3)
# s_score_out = np.round(np.arange(2,-2.1,-0.1),1)

# param1,param2=strat.optimize(2,DATA['H4']['EURUSD']['close'],s_score_value['EURUSD']['adjusted_s_scores_vol'],3,stop_loss,s_score_out)
# strat.set_params_strategie(param1,2.5,param2,0.001,s_score_value,5,20,'H4')
# # param3,param4=strat.optimize(2,DATA['H4']['EURUSD']['close'],s_score_value['EURUSD']['adjusted_s_scores_vol'],1,ema_period,slope_range)
# # strat.set_params_strategie(param1,2.5,param2,param4,s_score_value,param3,20,'H4')
# strat.optimize(2,DATA['H4']['EURUSD']['close'],s_score_value['EURUSD']['adjusted_s_scores_vol'],2,s_score,None)

NameError: name 'Strategie' is not defined